In [1]:
!pip install transformers accelerate sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
!pip install langchain chromadb langchain-community sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 97.7 MB/s eta 0:00:0

In [3]:
#Upload Knowledge Base
from google.colab import files
uploaded = files.upload()

Saving blog_data.txt to blog_data.txt
Saving chroma.sqlite3 to chroma.sqlite3
Saving PDF1.pdf.txt to PDF1.pdf.txt
Saving PDF2.pdf.txt to PDF2.pdf.txt
Saving PDF3.pdf.txt to PDF3.pdf.txt
Saving wikipedia_data.txt to wikipedia_data.txt


In [4]:
wiki_text = open("wikipedia_data.txt").read()
blog_text = open("blog_data.txt").read()
pdf1 = open("PDF1.pdf.txt").read()
pdf2 = open("PDF2.pdf.txt").read()
pdf3 = open("PDF3.pdf.txt").read()

# Combine everything
combined_text = wiki_text + "\n" + blog_text + "\n" + pdf1 + "\n" + pdf2 + "\n" + pdf3

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.split_text(combined_text)
docs = [Document(page_content=chunk) for chunk in chunks]

print("Total chunks created:", len(docs))

Total chunks created: 901


In [6]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Clear previous db if needed: delete folder or rename path
db = Chroma.from_documents(docs, embedding, persist_directory="/content")
db.persist()

print("Chroma DB rebuilt successfully!")

<ipython-input-6-71b9bc0ae783>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Chroma DB rebuilt successfully!


<ipython-input-6-71b9bc0ae783>:8: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [7]:
results = db.similarity_search("What is ADAS?", k=3)
for r in results:
    print("\n>>", r.page_content[:500])


>> 2  | Advanced Driver Assistance Systems (ADAS) Guide 2015	
Texas Instruments
Advanced driver assistance (ADAS)
Table of contents
Texas Instruments commitment 
to automotive safety 
Advanced driver assistance
systems (ADAS) are one of the 
fastest-growing application areas in 
vehicles today as ADAS features are 
incorporated into even the middle
and lower ends of the car market. A 
multitude of features can now warn 
drivers, allow better visibility into what

>> Texas Instruments	
Advanced Driver Assistance Systems (ADAS) Guide 2015  |  3
Advanced driver assistance (ADAS)
Overview
Today most advanced driver assistance system (ADAS) 
functions are basically their own independent system, with 
radar, camera and ultrasound the most common sensor types. 
Going forward, TI sees an increasing trend to use multiple, 
different sensor inputs and combine them to make more  
accurate decisions and identify critical situations.
Vision

>> Automated driving system
SAE identified 6 levels for 

In [8]:
queries = [
    "What sensors are used in ADAS?",
    "What is the future of autonomous driving?",
    "Explain sensor fusion in ADAS",
    "What are SAE levels of automation?"
]

for q in queries:
    print(f"\n Query: {q}")
    results = db.similarity_search(q, k=2)
    for r in results:
        print("→", r.page_content[:400])


 Query: What sensors are used in ADAS?
→ Sensors
→ 18  |  Advanced Driver Assistance Systems (ADAS) Guide 2015	
Texas Instruments
Sensor fusion
Overview
Advanced driver assistance systems (ADAS) are still treated 
as separate systems, independent from each other. Each 
system has its own purpose and either displays information 
or performs an activity (such as a chime) without 
consideration for any other ADAS systems. Depending on 
the type of se

 Query: What is the future of autonomous driving?
→ From 2016 to 2018, the European Commission funded development for connected and automated driving through Coordination Actions CARTRE and SCOUT programs. The Strategic Transport Research and Innovation Agenda (STRIA) Roadmap for Connected and Automated Transport was published in 2019.
In November 2017, Waymo announced testing of autonomous cars without a safety driver. However, an employee was in 
→ In the 2020s
In 2022, a survey reported that only a quarter (27%) of the world's population

In [9]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
The token `sanjan` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate w

In [10]:
import os
os.environ["HF_TOKEN"] = "#Enter your hugging face token"

In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load the model and tokenizer
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto",
      token=os.environ["HF_TOKEN"]
)

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [12]:
automotive_keywords = [
    # ADAS & Autonomous
    "ADAS", "autonomous", "self-driving", "sensors", "LiDAR", "radar", "collision avoidance",
    "lane keep", "blind spot", "adaptive headlights", "traffic jam assist", "autonomous emergency braking",

    # Vehicle Systems
    "engine", "transmission", "braking", "powertrain", "suspension", "chassis", "steering",
    "ECU", "OBD", "TPMS", "torque", "horsepower", "traction control", "stability control",

    # Diagnostics & Maintenance
    "diagnostics", "error codes", "DTC codes", "OBD scanner", "dashboard warning lights",
    "predictive maintenance", "service interval", "maintenance schedule", "repair shop", "mechanic", "garage",

    # Repairs & Troubleshooting
    "DIY car repair", "car troubleshooting", "oil change", "brake pads replacement", "tire pressure check",
    "jump start", "coolant top-up", "brake fluid flush", "radiator flush", "fuse box", "battery replacement",
    "replace spark plugs", "change air filter", "belt squeal", "engine overheating", "engine misfire",
    "engine noise", "fluid leaks", "wiper blade replacement", "headlight bulb change",
    "clean MAF sensor", "throttle body cleaning", "timing belt", "timing chain", "alternator",
    "starter motor", "fuel pump", "water pump", "thermostat", "catalytic converter", "muffler",
    "struts", "shocks", "serpentine belt", "wheel alignment", "tire rotation", "battery check", "battery voltage",

    # Infotainment & Connectivity
    "infotainment system", "Bluetooth", "USB port", "touchscreen", "voice command", "Wi-Fi hotspot",
    "app integration", "software update", "firmware update", "connected car", "navigation", "GPS",
    "screen not working", "Bluetooth not pairing", "reset infotainment",

    # Comfort & Controls
    "air conditioning", "AC not cooling", "heater issue", "heated seats", "ventilated seats", "sunroof",
    "power windows", "keyless entry", "remote start", "interior lighting", "ambient lighting",
    "seat adjustment", "power mirrors", "rear defrost", "windshield wipers", "wiper motor",

    # Buying, Ownership & Legal
    "VIN", "Vehicle Identification Number", "financing", "leasing", "insurance", "registration", "title",
    "resale value", "trade-in", "used car", "certified pre-owned", "car loan", "odometer",
    "ownership cost", "vehicle history report", "warranty", "recall", "inspection checklist",

    # Vehicle Types
    "sedan", "SUV", "truck", "pickup", "coupe", "convertible", "hatchback", "minivan", "wagon",

    # Electric, Hybrid, Fuel
    "electric vehicle", "battery", "hybrid", "plug-in hybrid", "hydrogen vehicle",
    "battery electric vehicle", "fuel cell vehicle", "EV", "charging", "EV charging",
    "charging station", "Level 1 charging", "Level 2 charging", "DC fast charging",
    "charging connector", "charging speed", "range anxiety",

    # Fuel & Performance
    "fuel efficiency", "petrol", "diesel", "fuel economy", "fuel gauge", "fuel tank", "MPG", "MPGe",
    "kWh", "regenerative braking", "acceleration", "towing capacity", "handling", "drive modes",
    "Eco mode", "Sport mode", "off-road", "4WD", "AWD", "FWD", "RWD", "hill start assist", "parking assist",
    "backup camera", "360 camera", "reverse sensor",

    # Safety & Regulations
    "safety", "airbag", "seatbelt", "pretensioner", "crumple zone", "child seat", "ISOFIX",
    "pedestrian detection", "crash test", "safety rating", "emissions", "smog check", "inspection",
    "low emission zone", "fuel economy rating", "congestion charge", "EV incentives", "tax credit",

    # Connectivity & Future Tech
    "ride-sharing", "V2X", "over-the-air updates", "alternative fuel"
]


In [13]:
print(f"Total automotive keywords in the chatbot: {len(automotive_keywords)}")

Total automotive keywords in the chatbot: 198


In [14]:
#Keyword Filter Function
#This filters out irrelevant queries
def is_automotive_query(user_query):
    user_query_lower = user_query.lower()
    return any(keyword.lower() in user_query_lower for keyword in automotive_keywords)

In [15]:
#Fetch the context from ChromaDB
def retrieve_context(query, k=3):
    results = db.similarity_search(query, k=k)
    unique_chunks = list(dict.fromkeys(doc.page_content.strip() for doc in results))
    return "\n".join(unique_chunks)

In [17]:
# Final RAG + Mistral LLM Answer Generator
def generate_response(user_query):
    if not is_automotive_query(user_query):
        return "I'm designed to answer **technical automotive-related questions only**. Please ask about ADAS, vehicle technology, or car maintenance."

    # Step 1: Retrieve context from ChromaDB
    context = retrieve_context(user_query, k=3)

    # Step 2: Build the RAG prompt
    prompt = f"""
You are an expert automotive assistant.

Your goal is to provide **technically accurate**, **clear**, and **beginner-friendly** answers based on the context provided.

Avoid complex jargon unless it's necessary — and if you must use technical terms, explain them briefly.
Use simple analogies if it helps the user understand better.

Ensure the answer is **complete**, **well-structured**, and does **not end abruptly**.
Avoid giving short or stripped answers. Your response should read naturally, like a helpful expert explaining to a curious learner.

---
Context:
{context}

Question: {user_query}
Answer:
"""

    # Step 3: Mistral Inference with tuned generation settings
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
    output = model.generate(
        input_ids,
        max_new_tokens=512,
        temperature=0.5,
        top_p=0.7,
        do_sample=True,
        repetition_penalty=1.2,
        eos_token_id=tokenizer.eos_token_id  #Helps with graceful stopping
    )

    # Step 4: Decode and extract final answer
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    final_answer = response.split("Answer:")[-1].strip().split("Question:")[0].strip()

    return final_answer

In [18]:
test_query = "What sensors are used in ADAS?"
print("User:", test_query)
print("Bot:", generate_response(test_query))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


User: What sensors are used in ADAS?
Bot: Radar sensors are commonly used in advanced driver assistance systems (ADAS). They can detect objects at long range and work well in low light conditions. Radar sensors typically emit radio waves that bounce off objects in the environment and return to the vehicle. This allows the vehicle to determine the distance, speed, and direction of the object.

Camera sensors are also widely used in ADAS. They capture images of the surrounding environment and can be used to detect obstacles, pedestrians, and road signs. Camera sensors typically have high resolution and can see details down to a few centimeters. However, they may not work well in low light conditions or through foggy windows.

Ultrasound sensors are another type of sensor used in ADAS. They use sound waves to measure distances between objects. Ultrasound sensors typically operate at higher frequencies than radar sensors and can detect small objects at close range. However, they may not wo

In [19]:
test_query = "Why is my car overheating even after replacing the coolant?"
print("User:", test_query)
print("Bot:", generate_response(test_query))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


User: Why is my car overheating even after replacing the coolant?
Bot: There could be several reasons why your car is overheating even after replacing the coolant. Here are some possible causes and solutions:

1. Leakage in the cooling system: If there is a leakage in the cooling system, it can cause the engine to overheat. Check all hoses, belts, and connections for any signs of leaks. You may also want to flush the entire cooling system to remove any contaminants that might have accumulated over time.
2. Dirty radiator: A dirty radiator can restrict airflow, which can lead to overheating. Clean the radiator using a mixture of water and vinegar, or take it to a professional cleaner.
3. Clogged thermostat: The thermostat controls the flow of coolant through the engine. If it is clogged, it can prevent enough coolant from reaching the engine, causing it to overheat. Flush the thermostat or replace it if it is damaged.
4. Low coolant level: Make sure that the coolant level is sufficient.

In [20]:
test_query = "Who won the Champions league last year?"
print("User:", test_query)
print("Bot:", generate_response(test_query))

User: Who won the Champions league last year?
Bot: I'm designed to answer **technical automotive-related questions only**. Please ask about ADAS, vehicle technology, or car maintenance.


In [21]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 100.1 MB/s eta 0:00:00


In [25]:
import gradio as gr

# Define Gradio interface function
def gradio_chatbot_interface(user_input):
    return generate_response(user_input)

# Launch the UI
with gr.Blocks(title="DriveMate – Your Everyday Car Tech Companion") as demo:
    # Centered title using HTML
    gr.HTML("<h2 style='text-align: center;'>🚗 <b>DriveMate – Your Everyday Car Tech Companion</b></h2>")
    gr.Markdown("Ask me anything about ADAS, sensors, maintenance, and general car technology. I'm here to help you understand your vehicle better!")

    with gr.Column():
        user_input = gr.Textbox(label="Your Question", placeholder="e.g., How does adaptive cruise control work?")
        output_text = gr.Textbox(label="DriveMate's Answer", interactive=False)
        submit_btn = gr.Button("Submit")
        clear_btn = gr.Button("Clear")

    # Button actions
    submit_btn.click(fn=gradio_chatbot_interface, inputs=user_input, outputs=output_text)
    clear_btn.click(lambda: ("", ""), None, [user_input, output_text])

# Run the Gradio app
demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3c9f378607b137abb3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
